In [1]:
#Installing required package to run the Notebook (only needed the very first time of usage)
#!pip install ipywidgets matplotlib tqdm nodejs npm fabio
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
#!mkdir data results
!mkdir data/MOKE results/MOKE

In [26]:
#Initializing every widgets and variables needed to run XRD plots
import os, glob, time, fabio;
import matplotlib.pyplot as plt;
import numpy as np;
from ipywidgets import interactive, widgets, fixed, Layout, Button, GridBox;
from IPython.display import display;
from tqdm.notebook import tqdm;
from scipy.interpolate import griddata;
from scipy.spatial import QhullError;

#Widgets for raw and treated XRD data interactive plots
#3;SrsN3%g#3G4q
W_folderpath = widgets.Dropdown(
    options=[('Select a folder', None)]+[(elm, elm) for elm in os.listdir('./data/MOKE/') if not elm.startswith('.')],
    value=None,
    description='Folder',
    disabled=False,
    layout=Layout(width='auto', grid_area='path')
);
W_folderpath_2 = widgets.Dropdown(
    options=[('Select a folder', None)],
    value=None,
    description='Subfolder',
    disabled=False,
    layout=Layout(width='auto', grid_area='path2')
);
W_slider_x = widgets.IntSlider(
    min=-60, max=60,
    step=5,
    description='X position',
    layout=Layout(width='auto', grid_area='X')
);
W_slider_y = widgets.IntSlider(
    min=-40, max=40,
    step=5,
    description='Y position',
    layout=Layout(width='auto', grid_area='Y')
);
W_slider_Xrange = widgets.IntRangeSlider(
    value=[0, 100],
    min=0, max=100,
    step=1,
    description='Time',
    layout=Layout(width='auto', grid_area='Xrange')
);
W_slider_Yrange = widgets.FloatRangeSlider(
    value=[-.5, .5],
    min=-1, max=1,
    step=.01,
    description='Kerr rot.',
    layout=Layout(width='auto', grid_area='Yrange')
);
W_offset = widgets.FloatSlider(
    value=1.0,
    min=-5.0, max=5.0,
    step=0.1,
    description='Sum offset',
    layout=Layout(width='auto', grid_area='offset')
);
W_smooth = widgets.Checkbox(
    value=True,
    description='Smooth',
    tooltip='Apply or remove smooth function for data',
    disabled=False,
    indent=False,
    layout=Layout(width='auto', grid_area='smooth')
);
W_average = widgets.Checkbox(
    value=True,
    description='Average',
    tooltip='Average between the different aquisition data',
    disabled=False,
    layout=Layout(width='auto', grid_area='average')
);
W_avg_drop = widgets.Dropdown(
    options=[],
    value=None,
    description='Aquisition n°',
    disabled=True,
    layout=Layout(width='auto', grid_area='avg_drop')
);
W_bgr_type = widgets.Dropdown(
    options=[('None', 1)],
    value=1,
    description='Background',
    disabled=True,
    layout=Layout(width='auto', grid_area='bgr_type')
);
W_RR_out = widgets.Output(
    layout=Layout(width='auto', grid_area='RR_out')
);
W_map_file = widgets.Text(
    value = None,
    description='Filename',
    layout=Layout(width='auto', grid_area='map_file')
);
W_start_button = widgets.ToggleButton(
    value=False,
    description='Start',
    disabled=False,
    tooltip='Create a file containing all the refined parameters from Rietveld refinement',
    layout=Layout(width='auto', grid_area='start_button')
);
W_data_type = widgets.Dropdown(
    options=[()],
    value=None,
    description='Data to plot',
    disabled=False,
    layout=Layout(width='100%', grid_area='data_type')
);
W_plot_dim = widgets.Dropdown(
    options=[('2D map plot',1)],
    value=1,
    description='Plot type',
    disabled=True,
    layout=Layout(width='100%', grid_area='plot_dim')
);
W_data_file = widgets.Dropdown(
    options=[()],
    value=None,
    description='Datafile',
    disabled=False,
    layout=Layout(width='100%', grid_area='data_file')
);

W_tabs = widgets.Tab();
tab_names = ['Raw XRD data', 'Rietveld Refinement', 'Parameter Extraction', '2D Parameter Maps'];
W_tabs.children = [widgets.Text(description='filler1'),
                   widgets.Text(description='filler2'),
                   widgets.Text(description='filler3'),
                   widgets.Text(description='filler4')];
for i, elm in enumerate(tab_names):
    W_tabs.set_title(i, str(elm));
output = widgets.Output();

#defining all functions for the raw plots (1st tab)
def getBoundaries(folderpath, folderpath_2):
    if folderpath is not None and folderpath_2 is not None:
        fullpath = f'./data/MOKE/{folderpath}/{folderpath_2}/info.txt';
        try:
            with open(fullpath, 'r', encoding='iso-8859-1') as file_info :
                for line in file_info:
                    if 'Pulse_voltage' in line:
                        pulse_volt = line.split('=')[1];
                    elif 'Average_per_point' in line:
                        avg_pts = line.split('=')[1];
                    elif 'Number_of_points_x' in line:
                        nb_x = int(line.split('=')[1]);
                    elif 'Number_of_points_y' in line:
                        nb_y = int(line.split('=')[1]);
                    elif 'Scanning_dimension_x' in line:
                        length_x = float(line.split('=')[1]);
                    elif 'Scanning_dimension_y' in line:
                        length_y = float(line.split('=')[1]);
                    elif 'shape' in line:
                        shape = line.split('=')[1].strip();
                return pulse_volt, avg_pts, nb_x, nb_y, length_x, length_y, shape;
        except FileNotFoundError:
            print(f'info.txt file not found (required to plot data)');
            return None;
    return None;

def initialize_sliders(slider_x, slider_y, nb_x, nb_y, length_x, length_y, shape):
    if shape == 'CIRCLE':
        step = int(length_x/nb_x);
        max = int(step*(nb_x-1)/2); min = -max;

        W_slider_x.step, W_slider_y.step = step, step;
        W_slider_x.min, W_slider_y.min = min, min;
        W_slider_x.max, W_slider_y.max = max, max;
        return min, min, max, max, step, step;
    elif shape == 'RECTANGULAR':
        step_x = int(length_x/(nb_x-1)); step_y = int(length_y/(nb_y-1));
        max_x = int(step_x*(nb_x-1)/2); min_x = -max_x;
        max_y = int(step_y*(nb_y-1)/2); min_y = -max_y;
        
        W_slider_x.step, W_slider_y.step = step_x, step_y;
        W_slider_x.min, W_slider_y.min = min_x, min_y;
        W_slider_x.max, W_slider_y.max = max_x, max_y;
        return min_x, min_y, max_x, max_y, step_x, step_y;
    else:
        print('not rect or circle');
        return None;
    return 0;

def createFilename(folderpath, folderpath_2, slider_x, slider_y):
    '''converting coordinates to indexes in Areamap00x00y.ext'''
    if folderpath is None or folderpath_2 is None:
        print(f"Start by selecting a folder with your data.");
        return None;
    fullpath = f'./data/MOKE/{folderpath}/{folderpath_2}/';
    
    mag = f'x{float(slider_x):.1f}_y{float(slider_y):.1f}_magnetization.txt';
    pul = f'x{float(slider_x):.1f}_y{float(slider_y):.1f}_pulse.txt';
    sum = f'x{float(slider_x):.1f}_y{float(slider_y):.1f}_sum.txt';
    data_mag = f'{fullpath}/{mag}';
    data_pul = f'{fullpath}/{pul}';
    data_sum = f'{fullpath}/{sum}';

    count_check = 0;
    for file in os.listdir(fullpath):
        if count_check == 3:
            break;
        elif file.endswith(mag):
            data_mag = f'{fullpath}/{file}';
            count_check += 1;
        elif file.endswith(pul):
            data_pul = f'{fullpath}/{file}';
            count_check += 1;
        elif file.endswith(sum):
            data_sum = f'{fullpath}/{file}';
            count_check += 1;
    print(data_mag, data_pul, data_sum)
    return 0;

def getRangeType(range_type, wavelength):
    match range_type:
        case 1:
            factor = lambda a : a;#f(x) = x since we already have 2theta in the datafile
            current_max, current_step, current_description = 90, 1, '2Theta';
            x_label = f'2Theta (°)';
        case 2:
            pi = np.arccos(-1);
            factor = lambda a : 4*pi*np.sin(a*pi/180)/wavelength;#f(x) = 4*pi*sin(2x/2)/lambda (scattering vector Q)
            x_label = f'Wavevector Q (Å^-1)';
            current_max, current_step, current_description = 30, 0.25, 'Vector Q';
    W_slider_Xrange.max, W_slider_Xrange.step, W_slider_Xrange.description = current_max, current_step, current_description;
    return factor, x_label;

def getXrdType(xrd_type):
    '''defining the prefix and suffix of the datafile depending if it is Smartlab (case 1), ESRF (case 2) or custom (case 3)'''
    wavelength = 1.54060;#in Å (or 8.04 keV, CuKalpha)
    
    match xrd_type:
        case 1:
            W_prefix.value = 'Areamap'; W_suffix.value = 'ras';
            prefix = W_prefix.value; suffix = W_suffix.value;
            W_prefix.disabled = True; W_suffix.disabled = True;
        case 2:
            W_prefix.value = 'ESRF_Areamap'; W_suffix.value = 'dat';
            prefix = W_prefix.value; suffix = W_suffix.value;
            W_prefix.disabled = True; W_suffix.disabled = True;
            wavelength = 0.61992;#in Å (or 20 keV)
        case 3:
            W_prefix.disabled = False; W_suffix.disabled = False;
            prefix = W_prefix.value; suffix = W_suffix.value;
            #wavelength = 0.61992;#in Å (or 20 keV)
            wavelength = 1.54060;#in Å (or 8.04 keV, CuKalpha)
    return prefix, suffix, wavelength;

def readFromFile(datafile, slider_x, slider_y):
    x_values, y_values = [], [];
    try:
        with open(datafile, 'r', encoding='iso-8859-1') as file:
            for line in file:
                if not line.startswith('*'):#skipping the header (but contains useful info so might save some of the lines in a file?)
                    data = line.split();
                    x_values.append(float(data[0]));
                    y_values.append(float(data[1]));
    except FileNotFoundError:
        print(f'File {datafile} not found at position ({slider_x}, {slider_y})');
        return None;
    return x_values, y_values;

def display2DImg(enable_img, datafile):
    if enable_img:
        img_lst = glob.glob(f"{datafile.split('.ras')[0]}*.img");
        if len(img_lst) > 0:
            image = fabio.open(img_lst[0]);
            return image;
    return None;

def addToGraph(folderpath, slider_x, slider_y, x_values, y_values):
    global SAVE_PLOT_MULT;
    list_append = [folderpath, slider_x, slider_y, x_values, y_values];
    
    if (list_append in SAVE_PLOT_MULT):
        return 1;
    SAVE_PLOT_MULT.append(list_append);
    return 0;

def removeFromGraph(folderpath, slider_x, slider_y):
    global SAVE_PLOT_MULT;
    TMP_LIST = [sub for sub in SAVE_PLOT_MULT if (sub[0] != folderpath or sub[1] != slider_x or sub[2] != slider_y)];
    SAVE_PLOT_MULT = TMP_LIST;
    return 0;

def checkSaveState(folderpath, prefix, suffix, x_values, y_values, slider_x, slider_y,
                   add, remove, remove_all, add_allX, add_allY,
                   start_x, start_y, end_x, end_y, step_x, step_y):
    '''check the different button state to save/remove plot from saved plot list'''
    global SAVE_PLOT_MULT;
    #if reset, we empty the saved list of plots.
    if remove_all == True :
        SAVE_PLOT_MULT = [];
        return 1;
        
    if len(SAVE_PLOT_MULT) > 0:
        counter = 0;
        for line in SAVE_PLOT_MULT:
            if (line[0] == folderpath and line[1] == slider_x and line[2] == slider_y):
                W_add.value = False;
                W_remove.value = False;
                W_add.disabled = True;
                W_remove.disabled = False;
                counter = 1;
        if counter == 0 :
            W_add.value = False;
            W_remove.value = False;
            W_add.disabled = False;
            W_remove.disabled = True;
            
    if (add == True):
        addToGraph(folderpath, slider_x, slider_y, x_values, y_values);
        return 1;
    if (remove == True):
        removeFromGraph(folderpath, slider_x, slider_y);
        return 1;
    if (add_allX == True):
        x = start_x;
        while (x <= end_x):
            datafile = createFilename(folderpath, prefix, suffix, x, slider_y, start_x, start_y, step_x, step_y, 'ras');
            save_x_values, save_y_values = readFromFile(datafile, x, slider_y);
            addToGraph(folderpath, x, slider_y, save_x_values, save_y_values);
            x += step_x;
        return 1;
    if (add_allY == True):
        y = start_y;
        while (y <= end_y):
            datafile = createFilename(folderpath, prefix, suffix, slider_x, y, start_x, start_y, step_x, step_y, 'ras');
            save_x_values, save_y_values = readFromFile(datafile, slider_x, y);
            addToGraph(folderpath, slider_x, y, save_x_values, save_y_values);
            y += step_y;
        return 1;
    return 0;

def setButtonsValues(add, remove, remove_all, add_allX, add_allY, plt):
    if remove_all == True :
        W_add.disabled = False;
        W_remove.disabled = True;
        W_remove_all.value = False;
        return 0;         
    if (add == True):
        W_add.disabled = True;
        W_remove.disabled = False;
        W_add.value = False;
        return 0;
    if (remove == True):
        W_remove.disabled = True;
        W_add.disabled = False;
        W_remove.value = False;
        plt.close('all')
        return 0;
    if (add_allX == True):
        W_add_allX.value = False;
        return 0;
    if (add_allY == True):
        W_add_allY.value = False;
        return 0;
    return 0;

def getOffset(offset, offset_all):
    offset_all_value = 0;
    if offset_all:
        offset_all_value = offset;
        offset = 0;
    if offset_all_value < 0 :
        W_slider_Yrange.min, W_slider_Yrange.max = -100000, 10000;
    elif offset_all_value > 0 or not offset_all:
        W_slider_Yrange.min, W_slider_Yrange.max = -10000, 100000;
    return offset, offset_all_value;

def readFromDia(dia_file):
    LIST_VALUES = [];#list containing all the data (2theta, I_exp, I from different phases)
    try:
        with open(dia_file, 'r', encoding='iso-8859-1') as file:
            #getting informations from header
            header = next(file);
            header_spl = header.split();
            global phases;
            phases = header_spl[8:];#getting the nb of phases and phase names to add in legend plot

            #creating list for each different phases found during the refinement to plot them individually
            for i in range(4+len(phases)):
                LIST_VALUES.append([]);
            
            for line in file:
                data = line.split();
                #adding each curve to a list, LIST_VALUES contains all of the curves as [[curve1], [curve2], ect...]
                for j, LIST in enumerate(LIST_VALUES):
                    LIST.append(float(data[j]));
        #calculating the error and adding it to the plot list
        offset = 2000;#moving the error curve to -offset value
        ERROR = [LIST_VALUES[1][i] - LIST_VALUES[2][i] - offset for i, theta in enumerate(LIST_VALUES[0])];#error=calc-exp-offsetERROR);
        LIST_VALUES.append(ERROR);
        return LIST_VALUES;
    except FileNotFoundError:
        return None;

def readFromLst(lst_file, slider_x, slider_y):
    try :
        with open(lst_file, 'r', encoding='iso-8859-1') as file:
            global header_lst;
            header_lst = 'x_pos\ty_pos\t';#dynamically creating an header for the final datafile to plot refined parameter maps
            DATA_RR_OUTPUT = [slider_x, slider_y]; FIT_RR_OUTPUT = [];
        
            #reading the .lst file line by line to get lattice parameters
            current_phase = None;
            for line in file:
                if line.startswith('Rp='):
                    R_factors = line.split('  ')[-1].split(' ');
                    for elm in R_factors:
                        FIT_RR_OUTPUT.append(elm.strip());
                elif line.startswith('Local parameters and GOALs for phase'):
                    current_phase = line.split()[-1];#name of the current phase for the refined lattice parameters
                    FIT_RR_OUTPUT.append(current_phase);
                elif line.startswith('A=') or line.startswith('C=') or line.startswith('B='):
                    FIT_RR_OUTPUT.append(line.strip());
                    line_lattice=line.split('=');#the line is in this format 'C=1.234567+-0.001234' and we split it to only get the value and err
                    letter = line_lattice[0];
                    if '+-' in line :
                        lattice = line_lattice[1].split('+-');
                    else :
                        lattice = [line_lattice[1].rstrip(), '0.000000'];
                    header_lst += f'{current_phase}_{letter}\t{current_phase}_{letter}_err\t';#adding the column name to the header
                    DATA_RR_OUTPUT.append(lattice[0]);
                    DATA_RR_OUTPUT.append(lattice[1].rstrip());
        
                #extracting volume fractions now (but is actually found first in the .lst file)
                else :
                    for k, elm in enumerate(phases):
                        elmt = elm.split('=')[1];
                        #same stuff, getting volume fraction values, format can be 'QNd2Fe14B=0.456789' for example
                        if line.startswith('Q'+elmt):
                            FIT_RR_OUTPUT.append(line.strip())
                            if '+-' in line :
                                fraction = line.split('=')[1].split('+-');
                            else :
                                fraction = [line.split('=')[1].rstrip(), '0.000000'];
                            header_lst += f'Q{elmt}\tQ{elmt}_err\t';
                            DATA_RR_OUTPUT.append(fraction[0]);
                            DATA_RR_OUTPUT.append(fraction[1].rstrip());
        return DATA_RR_OUTPUT, FIT_RR_OUTPUT;
    except FileNotFoundError:
        return None;
        
def raw_plot(folderpath, folderpath_2, average, avg_drop,
             slider_x, slider_y, smooth,
             slider_Xrange, slider_Yrange, offset):
    '''Plotting function checking the state of every widget and plot the corresponding graph'''
    global SAVE_PLOT_MULT;
    try:
        pulse_volt, avg_pts, nb_x, nb_y, length_x, length_y, shape = getBoundaries(folderpath, folderpath_2);
        initialize_sliders(slider_x, slider_y, nb_x, nb_y, length_x, length_y, shape);
        #fetching all the inputs from user
    except TypeError:
        print("Select a folder to start");
        return 1;
    createFilename(folderpath, folderpath_2, slider_x, slider_y);
    plt.close('all');
    
    #plotting the figures
    return 0;

def rietveld_plot(folderpath, XRD_type, slider_x, slider_y, slider_Xrange, slider_Yrange):
    try :
        #fetching all the inputs from user
        prefix, suffix, wavelength = getXrdType(XRD_type);
        start_x, start_y, end_x, end_y, step_x, step_y = getBoundaries(folderpath, prefix, suffix);
        dia_file, lst_file = createFilename(folderpath, prefix, suffix, slider_x, slider_y, start_x, start_y, step_x, step_y, 'dia');
        LIST_VALUES = readFromDia(dia_file);
        DATA_RR_OUTPUT, FIT_RR_OUTPUT = readFromLst(lst_file, slider_x, slider_y);
    except TypeError:
        print("Select a folder to start");
        return 1;
    if (LIST_VALUES is None or DATA_RR_OUTPUT is None):
        print(f'File {dia_file} not found at position ({slider_x}, {slider_y})');
        return 1;
    #setting visuals and legend for the plot
    colortab=['g', 'r', 'b', 'purple', 'black', 'grey', 'c', 'm', 'y', 'orange'];
    labeltab=['exp', 'calc', 'bgr'];
    name_phases = [phs.split('=')[1] for phs in phases];
    for phs in name_phases :
        labeltab.append(phs);
    labeltab.append('diff');
    #plotting the curves after rietveld refinement
    plt.figure(figsize=(15, 8));
    for i in range(1, len(LIST_VALUES)):
        plt.plot(LIST_VALUES[0], LIST_VALUES[i], linewidth = 0.9, color=colortab[i-1], label=labeltab[i-1]);
    plt.xlabel('2theta (deg)');
    plt.ylabel('Intensity (counts)');
    plt.xlim(slider_Xrange);
    plt.ylim(slider_Yrange);
    plt.title(f"Rietveld Refinement for {folderpath}");
    plt.legend();
    plt.grid(True);
    plt.figure(dpi=300);
    plt.show();

    W_RR_out.clear_output();
    with W_RR_out:
        global_parm = 0;
        for i, elm in enumerate(FIT_RR_OUTPUT):
            if 'Rwp' in elm:
                print(f'{elm}  {FIT_RR_OUTPUT[i+1]}\n');
            elif 'Rexp' in elm:
                continue;
            elif 'Q' in elm:
                if global_parm == 0:
                    global_parm = 1;
                    print(f'Global parameters\n**********************');
                    global_list = [y for y in FIT_RR_OUTPUT if y.startswith('Q')];
                    for elm in global_list:
                        print(f'{elm}');
                else:
                    continue;
            elif elm in name_phases:
                print(f'\n{elm} parameters\n**********************');
            else:
                print(elm);
    return 0;

def extract_param(folderpath, map_file, save_button, prefix, suffix):
    filename = W_map_file.value;
    if filename in os.listdir('./results/XRD/'):
        print(f'{filename} already exists !');
        return 0;
    if save_button:
        W_start_button.value = False;
        
        DATA_LIST = [];
        data_save_path = './results/XRD/' + filename;
        #getting all the datafile names of the folder
        lst_file_list = sorted(glob.glob(f"./data/XRD/{folderpath}/*{prefix}*.lst"));
        if len(lst_file_list) > 0:
            start_x, start_y, end_x, end_y, step_x, step_y = getBoundaries(folderpath, prefix, suffix);
            
            for lst_file in tqdm(lst_file_list):
                indexes = lst_file.split('/')[4].split('_')[1].split('.')[0];
                index_x = indexes[:3];
                index_y = indexes[3:];
                x_pos =  (int(index_x)-1)*step_x+start_x;
                y_pos =  (int(index_y)-1)*step_y+start_y;
                data_line = readFromLst(lst_file, x_pos, y_pos)[0];
                if np.abs(x_pos) + np.abs(y_pos) <= 60:
                    line_frc = "";
                    for i, elm in enumerate(data_line):
                        if elm == 'UNDEF':
                            elm = 0.0;
                        line_frc += (str(elm)+'\t');
                    #print(line_frc);
                    DATA_LIST.append(line_frc);
            
            global header_lst;
            #print(header_lst);
            with open(data_save_path, 'w') as file:
                file.write(f'{header_lst}\n');
                for line in DATA_LIST:
                    file.write(f'{line}\n');
            print(f'File created {data_save_path}');
        else:
            print("No .lst or .dia file found. Please make the refinement first !");
    else :
        print("Enter the name of your file.");
    return 0;

def plot_2D(folderpath, file_value, data_type, plot_dim):
    global header;
    try :
        file = f'./results/XRD/{W_data_file.options[file_value][0]}';
        print(file)
        with open(file, 'r') as datafile:
            tab_header = (next(datafile)).strip().split();
            if tab_header != header:
                header = tab_header;
                W_data_type.options=[(elm, i) for i, elm in enumerate(header) if i > 1 and i%2 == 0];
                if W_data_type.value!=2:
                    W_data_type.value=2;
                else:
                    W_data_type.value=None;
                    W_data_type.value=2;
                return 0;
            if data_type is None:
                return 0;
            plot_number = data_type;
            data = np.genfromtxt(datafile, delimiter='\t', skip_header=0);
    except (FileNotFoundError, TypeError):
        print("File containing refined parameters not found.")
        return 1;
    X_data = data[:, 0];
    Y_data = data[:, 1];
    Z_data = data[:, plot_number];
    Z_err_data = data[:, plot_number+1];
    #convert dataframes to numpy arrays
    X, Y, Z, Z_err = np.array([]), np.array([]), np.array([]), np.array([]);
    for i in range(len(X_data)):
        X = np.append(X, X_data[i]);
        Y = np.append(Y, Y_data[i]);
        Z = np.append(Z, Z_data[i]);
        Z_err = np.append(Z_err, Z_err_data[i]);
    match plot_dim:
        case 1:
            #create x-y points to be used in heatmap
            xi = np.linspace(X.min(), X.max(), 500);
            yi = np.linspace(Y.min(), Y.max(), 500);
                
            #interpolation between the points for 2D map
            try:
                zi = griddata((X, Y), Z, (xi[None,:], yi[:,None]), method='cubic');
                zi_err = griddata((X, Y), Z_err, (xi[None,:], yi[:,None]), method='cubic');
            except QhullError:
                print(f"Error: Data is not 2-dimensionnal, cannot create a map");
                return 1;
            # Create the contour plot
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            CS = ax1.contourf(xi, yi, zi, 30, cmap=plt.cm.rainbow, vmax=max(Z), vmin=min(Z));
            fig.colorbar(CS, ax=ax1);
            CS_err = ax2.contourf(xi, yi, zi_err, 30, cmap=plt.cm.rainbow, vmax=max(Z_err), vmin=min(Z_err));
            fig.colorbar(CS_err, ax=ax2);
            
            ax1.set_title(f'{header[plot_number]}');
            ax2.set_title(f'{header[plot_number+1]}');
                    
            plt.tight_layout();
            plt.figure(dpi=300);
            plt.show();
    return 0;
    
def redirectFromTab(folderpath, folderpath_2, average, avg_drop,
                    slider_x, slider_y, smooth,
                    slider_Xrange, slider_Yrange, offset):
    '''checking which tab is currently selected by the user'''
    #start_time = time.time();

    match W_tabs.selected_index:
        case 0 :
            raw_plot(folderpath, folderpath_2, average, avg_drop,
                     slider_x, slider_y, smooth,
                     slider_Xrange, slider_Yrange, offset);
        case 1 :
            print(1)
        case 2 :
            print(2)
        case 3 :
            print(3)
    #print(f"Execution time (system) is {time.time()-start_time:.3f} s");
    return 0;

#global variables
SAVE_PLOT_MULT = [];

#interactive function needed to check an input from the user
widget = interactive(
    redirectFromTab,
    folderpath=W_folderpath,
    folderpath_2=W_folderpath_2,
    average=W_average,
    avg_drop=W_avg_drop,
    slider_x=W_slider_x,
    slider_y=W_slider_y,
    smooth=W_smooth,
    slider_Xrange=W_slider_Xrange,
    slider_Yrange=W_slider_Yrange,
    offset=W_offset,
    continuous_update=False
    );

#defining the different layouts of each interactive plots
raw_plot_layout = Layout(
    width='100%',
    grid_gap='0px 0px',
    grid_template_rows='auto auto auto auto',
    grid_template_columns='16.66% 16.66% 16.66% 16.66% 16.66% 16.66%',
    grid_template_areas='''
    "path path X X Xrange Xrange"
    "path2 path2 Y Y Yrange Yrange"
    "average avg_drop smooth . offset offset"
    "plot plot plot plot plot plot"
    '''
);
rietveld_plot_layout = Layout(
    width='100%',
    grid_gap='0px 0px',
    grid_template_rows='auto auto auto',
    grid_template_columns='10% 6.66% 16.66% 16.66% 16.66% 16.66% 6.66% 10%',
    grid_template_areas='''
    "path path path X X Xrange Xrange Xrange"
    "XRD XRD XRD Y Y Yrange Yrange Yrange"
    "plot plot plot plot plot plot RR_out RR_out"
    '''
);
parameter_extraction_layout = Layout(
    width='100%',
    grid_template_rows='auto',
    grid_template_columns='20% 20% 20% 20%',
    grid_template_areas='''
    "map_file map_file start_button ."
    "plot plot plot plot"
    '''
);
map2D_layout = Layout(
    width='100%',
    grid_template_rows='auto auto',
    grid_template_columns='3% 30% 33% 30% 3%',
    grid_template_areas='''
    "plot_dim plot_dim data_type data_file data_file"
    ". plot plot plot ."
    '''
);
#defining the gridboxes
raw_gridbox = GridBox(
    children=[W_folderpath, W_folderpath_2, W_average, W_avg_drop,
              W_slider_x, W_slider_y, W_smooth,
              W_slider_Xrange, W_slider_Yrange, W_offset,
              widget.children[-1]],
    layout=raw_plot_layout);
widget.children[-1].layout=Layout(width='auto', grid_area='plot');

rietveld_gridbox = GridBox(
    children=[W_folderpath, W_folderpath_2, W_slider_x, W_slider_y, W_slider_Xrange, W_slider_Yrange, widget.children[-1]],
    layout=rietveld_plot_layout);

parameter_extraction = GridBox(
    children=[W_map_file, W_start_button, widget.children[-1]],
    layout=parameter_extraction_layout);

map2D_gridbox = GridBox(
    children=[W_plot_dim, W_data_type, W_data_file, widget.children[-1]],
    layout=map2D_layout);

#tab menu including all the different plots
tab_children = [raw_gridbox, rietveld_gridbox, parameter_extraction, map2D_gridbox];
W_tabs.children = tab_children;
display(W_tabs, output);

def on_tab_change(change):
    with output:
        index = W_tabs.selected_index;
        if index == 0:
            print(0)
        if index == 1:
            print(1)
        if index == 2:
            print(2)
        if index == 3:
            print(3)
                    
def on_folder_change(change):
    with output:
        if W_folderpath.value is not None:
            W_folderpath_2.options=[(elm, elm) for elm in os.listdir(f'./data/MOKE/{W_folderpath.value}') if not elm.startswith('.')];
        else:
            W_folderpath_2.options=[('Select a folder', None)];
            W_folderpath_2.value = None;
    return 0;
    
W_tabs.observe(on_tab_change, names='selected_index');
W_folderpath.observe(on_folder_change, names='value');

Output()